## basic ReAct(manual action)

In [1]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from openai import OpenAI
client = OpenAI(base_url= "https://api.lingyiwanwu.com/v1", api_key= "YOUR_API_KEY")


In [3]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="yi-medium", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content#str
        # return llm.invoke(
        #            input=self.messages)

In [4]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [5]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [6]:
abot = Agent(prompt)

In [7]:
result = abot("How much does a toy poodle weigh?")
result

'Thought: I should look up the average weight of a toy poodle using the average_dog_weight action.\nAction: average_dog_weight: Poodle (Toy)\nPAUSE'

In [8]:
print(result)

Thought: I should look up the average weight of a toy poodle using the average_dog_weight action.
Action: average_dog_weight: Poodle (Toy)
PAUSE


In [9]:
result = average_dog_weight("Toy Poodle")
result

'a toy poodles average weight is 7 lbs'

In [10]:
next_prompt = "Observation: {}".format(result)

In [11]:
abot(next_prompt)

'Answer: A toy poodle typically weighs around 7 lbs.'

In [12]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

## a little more complex question

if using 'yi-medium', it doesn't act as expect(the model itself will return a weight and caculate itself), but 'yi-large' will act as openai

In [33]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="yi-large",
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content#str
        # return llm.invoke(
        #            input=self.messages)

In [34]:
abot = Agent(prompt)

In [35]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

"Thought: To find the combined weight of the two dogs, I need to look up the average weight of each breed using the 'average_dog_weight' action and then add them together using the 'calculate' action.\n\nAction: average_dog_weight: Border Collie\nPAUSE"

In [36]:
print(abot.messages[-1]['content'])

Thought: To find the combined weight of the two dogs, I need to look up the average weight of each breed using the 'average_dog_weight' action and then add them together using the 'calculate' action.

Action: average_dog_weight: Border Collie
PAUSE


In [39]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [40]:
abot(next_prompt)

'Thought: Now that I have the average weight of a Border Collie, I need to find the average weight of a Scottish Terrier.\n\nAction: average_dog_weight: Scottish Terrier\nPAUSE'

In [41]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [42]:
abot(next_prompt)

'Thought: With the average weights of both breeds, I can now calculate their combined weight.\n\nAction: calculate: 37 + 20\nPAUSE'

In [43]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [44]:
abot(next_prompt)

'Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 pounds.'

## Add loop 

In [45]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

In [46]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ] 
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [47]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: To find the combined weight of the two dogs, I need to look up the average weight of each breed using the 'average_dog_weight' action, and then add them together using the 'calculate' action.

Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Now that I have the average weight of a Border Collie, I need to find the average weight of a Scottish Terrier using the 'average_dog_weight' action.

Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: With the average weights of both breeds, I can now calculate their combined weight.

Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.
